# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import io
import requests
import json

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.webservice import AciWebservice, LocalWebservice
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.model import Model


from azureml.widgets import RunDetails
from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.26.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

experiment_name = 'fraud_detection'

exp=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()
print(exp)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code APAEE33W8 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-142979
Azure region: southcentralus
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-142979
Experiment(Name: fraud_detection,
Workspace: quick-starts-ws-142979)


In [4]:
dataset = Dataset.get_by_name(workspace=ws, name='Banking-Transactions')
transactions_df = dataset.to_pandas_dataframe()

In [5]:
# Calculate the fraction of data points that are fraudulent
def fraudulent_percentage(transaction_df):
    '''Calculate the fraction of all data points that have a 'Class' label of 1; fraudulent.
       :param transaction_df: Dataframe of all transaction data points; has a column 'Class'
       :return: A fractional percentage of fraudulent data points/all points
    '''
    # counts for all classes
    counts = transactions_df['Class'].value_counts()
    
    # get fraudulent and valid cnts
    fraud_cnts = counts[1]
    valid_cnts = counts[0]
    
    # calculate percentage of fraudulent data
    fraud_percentage = fraud_cnts/(fraud_cnts+valid_cnts)
    
    return fraud_percentage

In [6]:
# call the function to calculate the fraud percentage
fraud_percentage = fraudulent_percentage(dataset)

print('Fraudulent percentage = ', fraud_percentage)
print('Total # of fraudulent pts: ', fraud_percentage*transactions_df.shape[0])
print('Out of (total) pts: ', transactions_df.shape[0])

Fraudulent percentage =  0.001727485630620034
Total # of fraudulent pts:  492.0
Out of (total) pts:  284807


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# Create compute cluster
# Choose a name for the computer cluster:
amlcompute_cluster_name = "compute-cluster"

# Check if the cluster already exists:
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('The cluster already exists')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
# Set the automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=aml_compute,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Class",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [9]:
# Submit the experiment
remote_run = exp.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
fraud_detection,AutoML_9150a662-0a53-42f6-b5b7-0239de5045d8,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
# Retrieve and save your best automl model.
best_automl_run, best_model = remote_run.get_output()
best_automl_model_name = best_automl_run.properties['model_name']
print(('Best AutoML model name:{}\n' \
       'Best AutoML model run:{}\n' \
       'Best AutoML model specification:{}').format(best_automl_model_name, str(best_automl_run), str(best_model)))

Best AutoML model name:AutoML9150a662038
Best AutoML model run:Run(Experiment: fraud_detection,
Id: AutoML_9150a662-0a53-42f6-b5b7-0239de5045d8_38,
Type: azureml.scriptrun,
Status: Completed)
Best AutoML model specification:Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               reg_alpha=0.5208333333333334,
                                           

In [12]:
registered_model = best_automl_run.register_model(model_name = 'best_automl_model', model_path = '/outputs/')
print(registered_model)

registered_model.download(target_dir='outputs', exist_ok=True)

Model(workspace=Workspace.create(name='quick-starts-ws-142979', subscription_id='976ee174-3882-4721-b90a-b5fef6b72f24', resource_group='aml-quickstarts-142979'), name=best_automl_model, id=best_automl_model:1, version=1, tags={}, properties={})


'outputs/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook142979/code/Users/odl_user_142979/outputs'

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

Creating a pipeline

In [31]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                memory_gb=4,
                                                enable_app_insights=True,
                                                description='predict a fraud')
model = Model(ws, 'best_automl_model')

env_deploy = Environment.get(workspace=ws, name='AzureML-AutoML')

inference_config = InferenceConfig(entry_script='score.py', environment=env_deploy)

service = Model.deploy(workspace=ws,
                       name='fraud-detection2',
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config)

In [32]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-18 03:46:53+00:00 Creating Container Registry if not exists.
2021-04-18 03:46:53+00:00 Registering the environment.
2021-04-18 03:46:53+00:00 Use the existing image.
2021-04-18 03:46:54+00:00 Generating deployment configuration.
2021-04-18 03:46:54+00:00 Submitting deployment to compute.
2021-04-18 03:46:58+00:00 Checking the status of deployment fraud-detection2..
2021-04-18 03:51:22+00:00 Checking the status of inference endpoint fraud-detection2.
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 2c88d08c-7e36-4553-80e8-e182b77a4665
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: fraud-detection2. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image viennaglobal.azurecr.io/azureml/azureml_39c362815a3a9e28eebe0c654b193f75 locally. Please refer to https://aka.ms/debugi

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 2c88d08c-7e36-4553-80e8-e182b77a4665
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: fraud-detection2. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image viennaglobal.azurecr.io/azureml/azureml_39c362815a3a9e28eebe0c654b193f75 locally. Please refer to https://ak

In [ ]:
print(service.scoring_uri)
print(service.swagger_uri)

Send a request to the web service deployed and test it 